# **Particle Swarm optimization**
class Particle:
- Define the particle object with the **__init__**. Every particle must have a position, a velocity.
- Define a function, called **FitnessCalculator**, that computes the fitness value of the particle. Its indipendent variable is the **accuracy metric** of the neural network.
- Define a function, called **PositionCalculator**
- Define a function, called **VelocityCalculator**


In [205]:
import numpy as np
import random
import particle

In [206]:
def InitializeSwarm(swarm_size, dimensionality, lower_bound, upper_bound):
    '''It takes as input the swarm size (number of particles I want to create, the dimensionality of the swarm (number of parameter for each particle) and the lower and upper bound of the parameter (that are two lists))'''

    #np.random.seed(3)
    # Usually the positions of particles are initialized to uniformly cover the search space
    swarm_list = []
    for particle in range(swarm_size):
        position = np.random.uniform(lower_bound, upper_bound, dimensionality)

        velocity = np.zeros(dimensionality)
        # velocity = np.random.random(dimensionality)

        part = Particle(position, velocity)
        swarm_list.append(part)

    return swarm_list

In [207]:
# Let's first define the function we want to use for the evaluation:

def f(lista):
    '''Definisco una funzione f che prende in input una lista di tre elementi (ovviamente va cambiata se cambio la swarm size)'''
    return(lista[0]**2+lista[1]**2+lista[2]**2)

In [208]:
def global_optimum(swarm):
    '''takes as input a swarm of particles and return the position of global optimum and the value of the fitness function in that global optimum'''

    global_opt = (min([particle.bestfit for particle in swarm]))
    best_global_position = swarm[np.argmin(np.array([particle.bestfit for particle in swarm]))].position

    return best_global_position, global_opt  

In [209]:
def acceleration_coefficient(lower_bound, upper_bound):
    '''It takes as input two vector: lower_bound is a vector containing the minimum element which can assume each parameter and upper_buond contains the maximum. e.g. for parameter j which assume value in range (aj,bj), the j-esimo element of lower_bound will be aj and the j-esimo element of upper_bund will be bj'''

    #random.seed(3)
    a1 = random.random()
    a2 = random.random()

    c1 = a1*(np.array(upper_bound)-np.array(lower_bound))
    c2 = a2*(np.array(upper_bound)-np.array(lower_bound))

    return c1,c2


In [210]:
def PSO_inizialization(swarm_size, dim, evaluation_funct, lower_bound, upper_bound):
    swarm = InitializeSwarm(swarm_size, dim, lower_bound, upper_bound)

    #First we need to evaluate each particle

    for particle in swarm:
        particle.FitnessCalculator(particle.position, evaluation_funct)

        # When we inizialize the swarm we need also to calculate the local optimum:
        particle.BestLocal(problem = 'minimum')

    return(swarm)

In [211]:
def PSO_alg(swarm, lower_bound, upper_bound, v_max, evaluation_funct):

    # First we calculate the starting global best position of the swarm
    best_global_position, global_opt = global_optimum(swarm)

    # Calculate the coefficient needed for updating the velocity of the particles.
    c1, c2 = acceleration_coefficient(lower_bound,upper_bound)

    #w = 0.9

    # set the termination criteria
    criteria_not_reach = True

    while criteria_not_reach:

        for particle in swarm:
            ## We need to calculate the new velocity:",
            particle.VelocityCalculator(c1, c2, best_global_position, v_max = v_max)

            ## Now we can calculate the new position:\n",
            particle.PositionCalculator(particle.velocity)

            ## We need to evaluate again the fitness function\n",
            particle.FitnessCalculator(particle.position, evaluation_funct)

            ## With the new position calculated we have to update the local best position:
            particle.BestLocal(problem = 'minimum')

            #print('New velocity: ', particle.velocity, '\n New position: ', particle.position, '\n New fitness: ', particle.fitness , '\n Best local position: ', particle.bestp, 'Best local fitness', particle.bestfit ,'\n')


            # w /= 2 #at each iteraton w is halved (IN THIS CASE I HAVE SET W TO A FIXED VALUE DIRECTLY INTO THE CLASS PARTICLE )

        # Then we update the global optimum
        best_position, opt = global_optimum(swarm)
        if opt< global_opt:
            global_opt = opt
            best_global_position = best_position
        
        # print('GLOBAL OPTIMUM: ', global_opt, 'GLOBAL OPT POSITION: ', best_global_position)

        # we set the criteria depending on the number of iteration
        if swarm[0].iteration == 200:
            criteria_not_reach = False
    
    return best_global_position, global_opt

In [212]:
def main():
    swarm_size = 10
    dim = 3
    evaluation_funct = f
    lower_bound=[0,0,0]
    upper_bound = [10,10,10]
    v_max = [3,3,3]

    swarm = PSO_inizialization(swarm_size, dim, evaluation_funct, lower_bound, upper_bound)

    best_global_position, global_opt = PSO_alg(swarm, lower_bound, upper_bound, v_max, evaluation_funct)
    
    print('Best position found: ', best_global_position, 'with an evaluation of: ',global_opt)

In [213]:
main()

Best position found:  [ 0.19178194  0.04518806 -0.48034884] with an evaluation of:  0.2695572831687413
